**1. Imports**

In [ ]:
pip install grad-cam

In [ ]:
import os
import numpy as np
import pickle

import torch
from torchvision  import transforms

from pytorch_grad_cam import GradCAM, HiResCAM

In [ ]:
# custom written code 
from xrays import create_datasets
from max_sensitivity import sample_eps_Inf, get_explanation, get_exp_sens, MaxSensitivity_Dataset, plot_scores_frequency, scores_boxplot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**2. Download from Kaggle**


In [ ]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download tawsifurrahman/covid19-radiography-database

100% 777M/778M [00:35<00:00, 24.5MB/s]
100% 778M/778M [00:35<00:00, 22.9MB/s]


In [ ]:
! unzip covid19-radiography-database.zip

**3. Datasets (need only test data for this notebook)**

In [ ]:
# to be applied to training data
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),          #recommended size
    transforms.RandomHorizontalFlip(),      #data augmentation
    transforms.RandomVerticalFlip(),        #data augmentation
    transforms.RandomRotation(degrees=20),  #data augmentation
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])
# to be applied to validation and test data
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])

In [ ]:
train_dataset, validation_dataset, test_dataset = create_datasets(train_transforms, test_transforms)

**4. Model**

In [ ]:
model_path = './drive/MyDrive/Colab_Notebooks/dataset_models/Covid-19_Radiography_Dataset/Models/'
resnet34 = torch.load(model_path + 'xrays_resnet34.pt')

In [ ]:
resnet34 = resnet34.cuda()
resnet34 = resnet34.eval()

**5. Max Sensitivity**

In [ ]:
def run_experiment(radius, labels, samples_per_img, xai):

    assert xai in ['gradcam', 'hirescam']

    if xai == 'gradcam':
      cam_instance = GradCAM(model=resnet34, target_layers=[resnet34.layer4[2].conv2], use_cuda=True)
    elif xai == 'hirescam':
      cam_instance = HiResCAM(model=resnet34, target_layers=[resnet34.layer4[2].conv2], use_cuda=True)
    
    print('radius:{} - labels:{}'.format(radius, labels))

    dataset = [(img, label) for (img, label) in test_dataset if (label in labels)]
    _, data_scores = MaxSensitivity_Dataset(dataset, resnet34, cam_instance, radius, samples_per_img)

    drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/Covid-19_Radiography_Dataset/Max_Sensitivity/exp_ResNet34/'

    if xai == 'gradcam':
      with open(drive_path + 'maxsens_resnet34_gradcam_radius:{}_labels:{}.pickle'.format(radius, labels), 'wb') as f:
        pickle.dump(data_scores, f)
    elif xai == 'hirescam':
      with open(drive_path + 'maxsens_resnet34_hirescam_radius:{}_labels:{}.pickle'.format(radius, labels), 'wb') as f:
        pickle.dump(data_scores, f)
    
    return data_scores

# >**GradCAM**

**radius 0.05**

In [ ]:
data_scores = run_experiment(radius=0.05, labels=[0,1,2,3], samples_per_img=20, xai='gradcam')

radius:0.05 - labels:[0, 1, 2, 3]
Total time: 790.7079713344574 secs
Correctly predicted images: 2017/2119
Avg secs per image:  0.39


In [ ]:
sum(data_scores)

172173.75260734558

**radius 0.1**

In [ ]:
data_scores = run_experiment(radius=0.1, labels=[0,1,2,3], samples_per_img=20, xai='gradcam')

radius:0.1 - labels:[0, 1, 2, 3]
Total time: 797.4243333339691 secs
Correctly predicted images: 2017/2119
Avg secs per image:  0.4


In [ ]:
sum(data_scores)

200439.44840240479

**radius 0.2**

In [ ]:
data_scores = run_experiment(radius=0.2, labels=[0,1,2,3], samples_per_img=30, xai='gradcam')

radius:0.2 - labels:[0, 1, 2, 3]
Total time: 1165.175552368164 secs
Correctly predicted images: 2017/2119
Avg secs per image:  0.58


In [ ]:
sum(data_scores)

216054.81696891785

**radius 0.3**

In [ ]:
data_scores = run_experiment(radius=0.3, labels=[0,1,2,3], samples_per_img=30, xai='gradcam')

radius:0.3 - labels:[0, 1, 2, 3]
Total time: 1154.234727859497 secs
Correctly predicted images: 2017/2119
Avg secs per image:  0.57


In [ ]:
sum(data_scores)

218065.8645324707

**radius 0.4**

In [ ]:
data_scores = run_experiment(radius=0.4, labels=[0,1,2,3], samples_per_img=40, xai='gradcam')

radius:0.4 - labels:[0, 1, 2, 3]
Total time: 1494.6067695617676 secs
Correctly predicted images: 2017/2119
Avg secs per image:  0.74


In [ ]:
sum(data_scores)

215732.6551513672

**radius 0.5**

In [ ]:
data_scores = run_experiment(radius=0.5, labels=[0,1,2,3], samples_per_img=40, xai='gradcam')

radius:0.5 - labels:[0, 1, 2, 3]
Total time: 1491.4784820079803 secs
Correctly predicted images: 2017/2119
Avg secs per image:  0.74


In [ ]:
sum(data_scores)

211808.62618637085

## >**HiResCAM**

**radius 0.05**

In [ ]:
data_scores = run_experiment(radius=0.05, labels=[0,1,2,3], samples_per_img=20, xai='hirescam')

radius:0.05 - labels:[0, 1, 2, 3]
Total time: 800.628368139267 secs
Correctly predicted images: 2017/2119
Avg secs per image:  0.4


In [ ]:
sum(data_scores)

127666.35947608948

**radius 0.1**

In [ ]:
data_scores = run_experiment(radius=0.1, labels=[0,1,2,3], samples_per_img=20, xai='hirescam')

radius:0.1 - labels:[0, 1, 2, 3]
Total time: 784.5959494113922 secs
Correctly predicted images: 2017/2119
Avg secs per image:  0.39


In [ ]:
sum(data_scores)

150460.5391960144

**radius 0.2**

In [ ]:
data_scores = run_experiment(radius=0.2, labels=[0,1,2,3], samples_per_img=30, xai='hirescam')

radius:0.2 - labels:[0, 1, 2, 3]
Total time: 1159.5973179340363 secs
Correctly predicted images: 2017/2119
Avg secs per image:  0.57


In [ ]:
sum(data_scores)

155508.26230049133

**radius 0.3**

In [ ]:
data_scores = run_experiment(radius=0.3, labels=[0,1,2,3], samples_per_img=30, xai='hirescam')

radius:0.3 - labels:[0, 1, 2, 3]
Total time: 1121.5242447853088 secs
Correctly predicted images: 2017/2119
Avg secs per image:  0.56


In [ ]:
sum(data_scores)

156517.77531051636

**radius 0.4**

In [ ]:
data_scores = run_experiment(radius=0.4, labels=[0,1,2,3], samples_per_img=40, xai='hirescam')

radius:0.4 - labels:[0, 1, 2, 3]
Total time: 1482.249017715454 secs
Correctly predicted images: 2017/2119
Avg secs per image:  0.73


In [ ]:
sum(data_scores)

156753.10398864746

**radius 0.5**

In [ ]:
data_scores = run_experiment(radius=0.5, labels=[0,1,2,3], samples_per_img=40, xai='hirescam')

radius:0.5 - labels:[0, 1, 2, 3]
Total time: 1468.2639546394348 secs
Correctly predicted images: 2017/2119
Avg secs per image:  0.73


In [ ]:
sum(data_scores)

155844.2975654602